In [1]:
import os, sys
sys.path.insert(1, "/".join(os.path.abspath('').split("/")[:-1]))

import torch

from transformer.layers.multi_head_attention.attention_mechanism.attn_params import CosformerParams, VanillaParams
from transformer.blocks import Block, HourglassBlock, TighteningBlock
from transformer.blocks.utils import ShiftRight, DownsamplingLayer, UpsamplingLayer

/Users/cezarygolecki/projects/magisterka/.venv/lib/python3.11/site-packages/rotary_embedding_torch/rotary_embedding_torch.py:35: FutureWarning: `torch.cuda.amp.autocast(args...)` is deprecated. Please use `torch.amp.autocast('cuda', args...)` instead.
  @autocast(enabled = False)
/Users/cezarygolecki/projects/magisterka/.venv/lib/python3.11/site-packages/rotary_embedding_torch/rotary_embedding_torch.py:268: FutureWarning: `torch.cuda.amp.autocast(args...)` is deprecated. Please use `torch.amp.autocast('cuda', args...)` instead.
  @autocast(enabled = False)


# Hourglass

In [5]:
batch_size, max_length, d_model = 8, 32, 64
num_heads = 4

decoder = HourglassBlock(n_layers=[4,2,4],
                         sizes=[32,16,32],
                         d_model=d_model,
                         num_heads=num_heads,
                         method_params=CosformerParams(),
                         apply_rotary_pos_enc=False,
                         dropout=0.1,
                         has_outproj=True,
                         act_fun=None,
                         post_norm= False,
                         downsampling_type="avg",
                         upsampling_type="linear",
                         attention_downsampling= False,
                         attention_upsampling= False,
                         upsampling_residual= True,
                         device='cpu'
                         )

x = torch.rand((batch_size, max_length, d_model))
decoder(x).shape

torch.Size([8, 32, 64])


torch.Size([8, 32, 64])

In [ ]:
# Encoder
batch_size, max_length, d_model = 8, 32, 64
num_heads = 4

encoder = Block(n_layers=3,
                  d_model=d_model,
                  num_heads=num_heads,
                  method_params=VanillaParams(),
                  apply_rotary_pos_enc=True,
                  dropout=0.1,
                  has_outproj=True,
                  act_fun=None,
                  norm_before=True,
                  device='cpu'
                  )

x = torch.rand((batch_size, max_length, d_model))
encoder(x).shape

# Tightening Block

In [4]:
batch_size, max_length, d_model = 8, 32, 64
num_heads = 4

tightening_block = TighteningBlock(n_layers=[2,2],
                                   sizes=[max_length, int(max_length/2)],
                          d_model=d_model,
                          num_heads=num_heads,
                          method_params=CosformerParams(),
                          apply_rotary_pos_enc=True,
                          dropout=0.1,
                          has_outproj=True,
                          act_fun=None,
                          device='cpu'
                          )

x = torch.rand((batch_size, max_length, d_model))
tightening_block(x).shape

torch.Size([8, 16, 64])